In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 199 kB 46.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a76216cdd1657a5f74fd9eb23634a2749640e23652d45cfb0a1530e868b60673
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("data-clean") \
    .getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
calendar = 'drive/MyDrive/Colab\ Notebooks/data/Calendar_cleaned.csv'
path = '/content/drive/MyDrive/Colab Notebooks/data/'

In [ ]:
df_calendar = spark.read.option("header",True).csv(calendar)
df_calendar.describe()

DataFrame[summary: string, listing_id: string, date: string, available: string, price: string]

In [ ]:
rdd_calendar = df_calendar.rdd

In [ ]:
rdd_whole = rdd_calendar.map(lambda x:(x.listing_id,x))

In [ ]:
type(rdd_whole.take(10)[0][1].available)

str

In [ ]:
def countListing(r):
  return (r[0],1)
def countOccupied(r):
  if r[1].available == 'False':
    return (r[0],(1,1))
  else:
    return (r[0],(0,1))

In [ ]:
rddCount = rdd_whole.map(countListing)

In [ ]:
rddCountOccuipied = rdd_whole.map(countOccupied)

In [ ]:
counts = rddCount.reduceByKey(lambda x,y:x+y)

In [ ]:
from pyspark import Row
occupied = rddCountOccuipied.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])).mapValues(lambda x:(x[0]/x[1])*100).map(lambda x:Row(listing=x[0],occupancy_rate=x[1]))

In [ ]:
occupied.collect()

[Row(listing='897387', occupancy_rate=0.0),
 Row(listing='898999', occupancy_rate=100.0),
 Row(listing='8571561', occupancy_rate=83.83561643835617),
 Row(listing='8461058', occupancy_rate=100.0),
 Row(listing='5310744', occupancy_rate=14.520547945205479),
 Row(listing='5195552', occupancy_rate=14.520547945205479),
 Row(listing='5163979', occupancy_rate=11.78082191780822),
 Row(listing='3020807', occupancy_rate=100.0),
 Row(listing='2434101', occupancy_rate=9.315068493150685),
 Row(listing='4244433', occupancy_rate=81.64383561643835),
 Row(listing='4512376', occupancy_rate=100.0),
 Row(listing='6068724', occupancy_rate=73.97260273972603),
 Row(listing='2299737', occupancy_rate=20.82191780821918),
 Row(listing='2210236', occupancy_rate=78.63013698630137),
 Row(listing='934398', occupancy_rate=100.0),
 Row(listing='941603', occupancy_rate=20.0),
 Row(listing='8461067', occupancy_rate=100.0),
 Row(listing='8508270', occupancy_rate=81.91780821917808),
 Row(listing='8510573', occupancy_rate=

In [ ]:
dfo = occupied.toDF()

In [ ]:
dfo.repartition(1).write.option("header",True).csv("occupancy_rate.csv")

In [ ]:
s_df = rdd_calendar.toDF()

In [ ]:
from pyspark.sql.functions import to_date,split
dfsplit = s_df.withColumn('datearray',split(s_df.date,'/'))
dfformat = dfsplit.withColumn('year',dfsplit.datearray[2])
dfformat = dfformat.withColumn('mm',dfsplit.datearray[1])
dfformat = dfformat.withColumn('dd',dfsplit.datearray[0])

In [ ]:
dfformat = dfformat.drop('date')
dfformat = dfformat.drop('datearray')

In [ ]:
from pyspark.sql.functions import concat_ws
dfdata = dfformat.withColumn('datestring',concat_ws('-',dfformat.year,dfformat.mm,dfformat.dd))

In [ ]:
dfdata = dfdata.withColumn('date',to_date(dfdata.datestring))

In [ ]:
dfdata = dfdata.drop('year')
dfdata = dfdata.drop('mm')
dfdata = dfdata.drop('dd')
dfdata = dfdata.drop('datestring')

In [ ]:
dfdata = dfdata.sort('date')

In [ ]:
dfdata.show(10)

+----------+---------+-----+----------+
|listing_id|available|price|      date|
+----------+---------+-----+----------+
|  30702338|    False|   80|2022-03-09|
|  30770595|    False|  500|2022-03-09|
|  30702414|    False|   40|2022-03-09|
|  30728278|    False|   60|2022-03-09|
|  30712148|    False| 4500|2022-03-09|
|  30595509|    False|   70|2022-03-09|
|  30716811|    False|  235|2022-03-09|
|  30519155|    False|   45|2022-03-09|
|  30719520|    False|   65|2022-03-09|
|  30539856|    False|   49|2022-03-09|
+----------+---------+-----+----------+
only showing top 10 rows



In [ ]:
dfdata.repartition(1).write.option("header",True).csv("listing_sort_2.csv")

In [ ]:
p_df = s_df.toPandas()

In [ ]:
import pandas as pd
compiledf = pd.read_csv('/content/listing_sort_2.csv/part-00000-7f1076fa-69b3-4b2a-b393-ae2afa2e122d-c000.csv',
    index_col='date',
    parse_dates=['date'])

In [ ]:
compiledf = compiledf.to_period()

In [ ]:
compiledf.head()

,listing_id,available,price
date,,,
2022-03-09,897387,True,130.0
2022-03-09,8571561,False,230.0
2022-03-09,8940979,False,230.0
2022-03-09,8193171,False,250.0
2022-03-09,5310744,False,100.0


In [ ]:
import numpy as np
compiledf['time'] = np.arange(len(compiledf.index))

In [ ]:
compiledf = compiledf.fillna(0)

In [ ]:
compiledf.head(10)

,listing_id,available,price,time
date,,,,
2022-03-09,897387,True,130.0,0
2022-03-09,8571561,False,230.0,1
2022-03-09,8940979,False,230.0,2
2022-03-09,8193171,False,250.0,3
2022-03-09,5310744,False,100.0,4
2022-03-09,5539544,False,70.0,5
2022-03-09,5572652,False,50.0,6
2022-03-09,5195552,False,70.0,7
2022-03-09,5163979,False,110.0,8


In [ ]:
from sklearn.linear_model import LinearRegression
X = compiledf.loc[:, ['time']]
y = compiledf.loc[:, 'price']
model = LinearRegression()
model.fit(X, y)
y_pred_1 = pd.Series(model.predict(X), index=X.index)

In [ ]:
y_pred_1

date
2022-03-09    255.885537
2022-03-09    255.885538
2022-03-09    255.885538
2022-03-09    255.885539
2022-03-09    255.885539
                 ...    
2023-03-30    259.948258
2023-03-30    259.948259
2023-03-30    259.948259
2023-03-30    259.948260
2023-03-30    259.948260
Length: 7625189, dtype: float64

In [ ]:
compiledf.head()

,listing_id,available,price,time
date,,,,
2022-03-09,897387,True,130.0,0
2022-03-09,8571561,False,230.0,1
2022-03-09,8940979,False,230.0,2
2022-03-09,8193171,False,250.0,3
2022-03-09,5310744,False,100.0,4


In [ ]:
compiledf['lag'] = compiledf['price'].shift(7)

In [ ]:
compiledf = compiledf.dropna()

In [ ]:
compiledf.head()

,listing_id,available,price,time,lag
date,,,,,
2022-03-09,5195552,False,70.0,7,130.0
2022-03-09,5163979,False,110.0,8,230.0
2022-03-09,5014943,False,120.0,9,230.0
2022-03-09,3265015,False,83.0,10,250.0
2022-03-09,4483704,False,185.0,11,100.0


In [ ]:
from sklearn.linear_model import LinearRegression
X = compiledf.loc[:, ['lag']]
y = compiledf.loc[:, 'price']  # create the target
y, X = y.align(X, join='inner')  # drop corresponding values in target

model = LinearRegression()
model.fit(X, y)

y_pred_2 = pd.Series(model.predict(X), index=X.index)

In [ ]:
y_pred_2

date
2022-03-09    257.785291
2022-03-09    257.888253
2022-03-09    257.888253
2022-03-09    257.908845
2022-03-09    257.754403
                 ...    
2023-03-30    258.475135
2023-03-30    257.950030
2023-03-30    258.423654
2023-03-30    257.790439
2023-03-30    257.744106
Length: 7625182, dtype: float64

In [ ]:
compiledf.head()

,listing_id,available,price,time,lag
date,,,,,
2022-03-09,897387,True,130.0,0,0.0
2022-03-09,8571561,False,230.0,1,130.0
2022-03-09,8940979,False,230.0,2,230.0
2022-03-09,8193171,False,250.0,3,230.0
2022-03-09,5310744,False,100.0,4,250.0


In [ ]:
rollingdf = compiledf.drop('listing_id',axis=1)

In [ ]:
rollingdf.head()

,available,price,time,lag
date,,,,
2022-03-09,True,130.0,0,0.0
2022-03-09,False,230.0,1,130.0
2022-03-09,False,230.0,2,230.0
2022-03-09,False,250.0,3,230.0
2022-03-09,False,100.0,4,250.0


In [ ]:
moving_average = compiledf.rolling(
    window=7,       # 365-day window
    center=True,      # puts the average at the center of the window
    min_periods=1,  # choose about half the window size
).mean()

In [ ]:
compiledf.take(10)

ValueError: ignored

In [ ]:
moving_average.take(10)

ValueError: ignored

In [ ]:
y_pred_1.to_csv('y_pred_1.csv')

In [ ]:
y_pred_2.to_csv('y_pred_2.csv')